This script generates the industrial facilities data used in the industry module. 

The following folders are directly used by the industry module (they can be pasted into the industry inputs folder). All of them contain sector subfolders.

1) extra_epa_frs_facilities_west: EPA FRS facilities that are not contained in the GHGRP
2) facilities_missing_combustion_data: EPA GHGRP facilities that are missing stationary fuel combustion emissions data
3) sector_breakdown_by_unit_fuel: A detailed breakdown of EPA GHGRP facilities with stationary fuel combustion emissions data into       individual facility units and the fuel types each unit consumes


The other output folders are for reference/data checking only.

Step 1: Import libraries and create objects/file paths

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path

import os

base_path = Path.cwd()

input_path = base_path / 'industry_processing_inputs' 
ghgrp_path = input_path / 'ghgp_data_2022.xlsx'

output_path = base_path / 'industry_processing_outputs'
output_path.mkdir(parents=True, exist_ok=True)

output_ghgrp_facilities_path = output_path / 'facilities_by_sector_2022'
output_ghgrp_facilities_gpkg_path = output_path / 'facilities_by_sector_2022_gpkg'

output_ghgrp_facilities_path.mkdir(parents=True, exist_ok=True)
output_ghgrp_facilities_gpkg_path.mkdir(parents=True, exist_ok=True)

naics_codes = [
    "331110", "331511", "3312", "3313", "327310", "325",
    "324110", "327211", "327212", "327213", "327215"
]

sector_to_naics = {'Iron_and_Steel': ["331110", "331511", "3312"], 'Aluminum': ['3313'], 'Cement': ['327310'], \
                    'Chemicals': ['325'], 'Refineries': ['324110'], 'Glass': ['327211', '327212', "327213", "327215"]}

Step 2: Read in the 2022 GHGRP facilities data, add inWestCensus and inWECC columns, filter 
for facilities that are in these regions, and output the facilities within each industrial sector.

In [2]:
facilities_df = pd.read_excel(ghgrp_path, header=3) 

# Convert to a GeoDataFrame 
facilities_gdf = gpd.GeoDataFrame(
    facilities_df,
    geometry=gpd.points_from_xy(facilities_df['Longitude'], facilities_df['Latitude']),
    crs='EPSG:4326'
)

# Add inWestCensus column
west_census = gpd.read_file(input_path / 'west_census_region.gpkg').to_crs(epsg=4326)
facilities_gdf['inWestCensus'] = gpd.sjoin(
    facilities_gdf, west_census[['geometry']],
    how='left', predicate='intersects'
)['index_right'].notna()


# Add inWECC column
wecc_zones = gpd.read_file(input_path / 'load_zones/load_zones.shp').to_crs(epsg=4326)
facilities_gdf['inWECC'] = gpd.sjoin(
    facilities_gdf, wecc_zones[['geometry']],
    how='left', predicate='intersects'
)['index_right'].notna()


# Keep only facilities in WECC or West Census
facilities_gdf = facilities_gdf[(facilities_gdf['inWECC']) | (facilities_gdf['inWestCensus'])]

# Save the NAICS Code as a string
facilities_gdf['Primary NAICS Code'] = facilities_gdf['Primary NAICS Code'].astype(str)

# Save the combined GPKG 
facilities_gdf.to_file(output_ghgrp_facilities_gpkg_path / 'ghgrp_west_census_and_wecc_facilities.gpkg', driver='GPKG')

# Output sector facilities

for sector in sector_to_naics.keys():
    sector_facilities = pd.DataFrame()
    for naics_code in sector_to_naics[sector]:
        naics_facilities = facilities_gdf[facilities_gdf['Primary NAICS Code'].str.startswith(str(naics_code))]
        sector_facilities = pd.concat([sector_facilities, naics_facilities])

    sector_facilities.to_file(output_ghgrp_facilities_gpkg_path / f'{sector}_facilities.gpkg', driver='GPKG')

    # Drop geometry before saving sector CSVs
    facilities_out = sector_facilities.drop(columns=['geometry'])
    sector_facilities.to_csv(output_ghgrp_facilities_path / f'{sector}_facilities.csv', index=False)

    print(f'Output successfully saved to {output_path / f"{sector}_facilities.csv"}')


Output successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs/Iron_and_Steel_facilities.csv
Output successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs/Aluminum_facilities.csv
Output successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs/Cement_facilities.csv
Output successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs/Chemicals_facilities.csv
Output successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs/Refineries_facilities.csv
Output successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs/Glass_facilities.csv


Step 3: Pair the GHGRP facility data with a more detailed breakdown of industrial units and fuel types used within each facility. 

Creates three outputs:
1) A folder with each industrial unit paired with its fuel type, facility, and emissions within each sector
2) A folder of .gpkg files with the GHGRP industrial facilities contained in the first folder within each sector
3) A folder of .gpkg files with the GHGRP industrial facilities missing from the first folder within each sector

In [3]:
# First, define paths and read in the files.

# Input paths
facilities_folder_path = output_path / 'facilities_by_sector_2022'
unit_fuel_path = input_path / 'emissions_by_unit_and_fuel_type_c_d_aa.xlsb'

# Output Paths
output_breakdown_path = output_path / 'sector_breakdown_by_unit_fuel'
output_breakdown_path.mkdir(parents=True, exist_ok=True)

output_unit_fuel_facilities_path = output_path / 'facilities_with_unit_fuel_2022'
output_unit_fuel_facilities_path.mkdir(parents=True, exist_ok=True)

output_missing_facilities_path = output_path / 'facilities_missing_combustion_data'
output_missing_facilities_path.mkdir(parents=True, exist_ok=True)

unit_fuel_data = pd.ExcelFile(unit_fuel_path)

unit_df = pd.read_excel(unit_fuel_data, sheet_name='UNIT_DATA', header=6)
fuel_df = pd.read_excel(unit_fuel_data, sheet_name='FUEL_DATA', header=5)


In [4]:
#==========
# Update cement Unit CO2 emissions, assuming that 45% of facility emissions are stationary combustion emissions
# and that these emissions are evenly distributed across each facility unit
#==========

# Make sure the facility CO2 column is correct
facilities_df = facilities_df.rename(columns={'CO2 emissions (non-biogenic) ': 'CO2 emissions (non-biogenic)'})

# Filter cement units in unit_df
cement_units_mask = unit_df['Primary NAICS Code'].astype('Int64').astype(str) == '327310'
cement_units = unit_df[cement_units_mask].copy()

# Get facility-level CO2 for cement facilities
cement_facilities_co2 = facilities_df[facilities_df['Primary NAICS Code'].astype('Int64').astype(str) == '327310'][['Facility Id', 'CO2 emissions (non-biogenic)']].copy()

cement_facilities_co2.rename(columns={'CO2 emissions (non-biogenic)': 'Facility_CO2_total'}, inplace=True)

# Merge facility totals into cement units
cement_units = cement_units.merge(cement_facilities_co2, on='Facility Id', how='left')

# Distribute 45% of facility CO2 evenly across units
unit_counts = cement_units.groupby('Facility Id')['Unit Name'].transform('count')
cement_units['Unit CO2 emissions (non-biogenic)'] = cement_units['Facility_CO2_total'] * 0.45 / unit_counts

# Create a mapping from (Facility Id, Unit Name) -> cement CO2
cement_co2_map = cement_units.set_index(['Facility Id', 'Unit Name'])['Unit CO2 emissions (non-biogenic)'].to_dict()

# Apply cement CO2 to unit_df, preserving non-cement units
unit_df['Unit CO2 emissions (non-biogenic)'] = unit_df.apply(
    lambda row: cement_co2_map.get((row['Facility Id'], row['Unit Name']), row['Unit CO2 emissions (non-biogenic)']),
    axis=1
)


In [5]:
# Remove rows that do not correspond to 2022
unit_df = unit_df[unit_df['Reporting Year'].astype(str) == '2022']
fuel_df = fuel_df[fuel_df['Reporting Year'].astype(str) == '2022']

# Merge the facility units DataFrame with the fuel breakdown DataFrame
merged = pd.merge(unit_df, fuel_df, on=['Facility Id', 'Facility Name', 'Unit Name', 'Primary NAICS Code'], how = 'left')
merged = merged[['Facility Id', 'Facility Name', 'Unit Name', 'Primary NAICS Code', 'Unit CO2 emissions (non-biogenic)', 'Specific Fuel Type']]

all_unit_fuels = pd.DataFrame()

# Merge the unit and fuel DataFrame with the facilities DataFrame, outputting a breakdown for each sector
for sector_file in os.listdir(facilities_folder_path):
    if '.csv' in sector_file:
        facilities_df = pd.read_csv(facilities_folder_path / sector_file)

        sector = sector_file.replace('_facilities.csv', '')
                
        output_df = pd.merge(facilities_df, merged, on='Facility Id', how='inner')

        output_df = output_df[['Facility Id', 'FRS Id', 'Facility Name_x', 'Latitude', 'Longitude', 'Unit Name', 'Primary NAICS Code_y', 'Unit CO2 emissions (non-biogenic)', 'Specific Fuel Type', 'inWECC', 'inWestCensus']]
        all_unit_fuels = pd.concat([all_unit_fuels, output_df])

        # Save the detailed results by facility unit and fuel
        output_df.to_csv(output_breakdown_path / f'{sector}_facilities_breakdown', index=False)

        # Save the facilities that have stationary combustion values in each sector
        facilities_with_unit_fuel = output_df.groupby('Facility Id').first().reset_index()
        facilities_with_unit_fuel = facilities_with_unit_fuel.drop(columns=['Unit CO2 emissions (non-biogenic)', 'Specific Fuel Type'])
        facilities_with_unit_fuel = gpd.GeoDataFrame(facilities_with_unit_fuel,  
            geometry=gpd.points_from_xy(facilities_with_unit_fuel['Longitude'], facilities_with_unit_fuel['Latitude']),
            crs='EPSG:4326')

        facilities_with_unit_fuel.to_csv(output_unit_fuel_facilities_path / f'{sector}_facilities.csv', index=False)
        facilities_with_unit_fuel.to_file(output_unit_fuel_facilities_path / f'{sector}_facilities.gpkg', driver='GPKG')

        print(f'Facility unit and fuel outputs successfully saved to {output_path}')
        print(f'Facility outputs successfully saved to {output_unit_fuel_facilities_path / f"{sector}_facilities.csv"}')


Facility unit and fuel outputs successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs
Facility outputs successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs/facilities_with_unit_fuel_2022/Refineries_facilities.csv
Facility unit and fuel outputs successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs
Facility outputs successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs/facilities_with_unit_fuel_2022/Aluminum_facilities.csv
Facility unit and fuel outputs successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs
Facility outputs successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs/facilities_with_unit_fuel_2022/Iron_

In [6]:
# Get a list of facilities that were in the facilities list but not in the unit fuels list with stationay combustion
stationary_combustion_facility_ids = all_unit_fuels['Facility Id'].astype(str).unique()

facilities_missing_combustion = facilities_gdf[~facilities_gdf['Facility Id'].astype(str).isin(stationary_combustion_facility_ids)]

for sector in sector_to_naics:
    sector_missing_facilities = pd.DataFrame()

    for naics in sector_to_naics[sector]:
        sector_missing_facilities = pd.concat([sector_missing_facilities, facilities_missing_combustion[facilities_missing_combustion['Primary NAICS Code'].astype(str).str.startswith(str(naics))]])

    sector_missing_facilities.to_csv(output_missing_facilities_path / f'{sector}_facilities.csv', index=False)
    sector_missing_facilities.to_file(output_missing_facilities_path / f'{sector}_facilities.gpkg', driver='GPKG')
    
    print(f'Output successfully saved to {output_missing_facilities_path / f"{sector}_facilities.csv"}')


Output successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs/facilities_missing_combustion_data/Iron_and_Steel_facilities.csv
Output successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs/facilities_missing_combustion_data/Aluminum_facilities.csv
Output successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs/facilities_missing_combustion_data/Cement_facilities.csv
Output successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs/facilities_missing_combustion_data/Chemicals_facilities.csv
Output successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_demand_model/pre_processing/industry_processing_outputs/facilities_missing_combustion_data/Refineries_facilities.csv
Output successfully saved to /Users/nicholaskong/Desktop/REAM_lab/h2_dema

Step 4: Create a list of EPA FRS facilities in the West Census Region for each sector that are not contained in the GHGRP. Add a column 'inWECC' to these 

In [7]:
# Read and prepare EPA FRS facilities. These facilities have already been filtered for the target sectors.
frs_list = pd.read_csv(input_path / 'epa_frs_facilities.csv')

# Keep only facilities with coordinates and registry_id
frs_coords = frs_list.dropna(subset=['longitude83', 'latitude83', 'registry_id']).copy()
frs_coords_gdf = gpd.GeoDataFrame(
    frs_coords,
    geometry=gpd.points_from_xy(frs_coords['longitude83'], frs_coords['latitude83']),
    crs="EPSG:4326"
)

# Filter for West Census Region
frs_west_gdf = gpd.sjoin(frs_coords_gdf, west_census, how='inner', predicate='intersects')
frs_west_gdf = frs_west_gdf.drop(columns=[col for col in frs_west_gdf.columns if col.endswith('_right')], errors='ignore')

# Flag facilities that exist in unit fuels (GHGRP) data
existing_frs_ids = all_unit_fuels.reset_index()['FRS Id'].dropna().astype('Int64').astype(str)

frs_west_gdf['has_existing_id'] = frs_west_gdf['registry_id'].astype(str).isin(existing_frs_ids)

# Group by primary_name: if any registry_id for a facility exists in GHGRP, mark all rows of that facility
facility_flags = (
    frs_west_gdf.groupby('primary_name')['has_existing_id']
    .transform('any')
    .fillna(False)
    .astype(bool)
)

# Keep only facilities not already in GHGRP/unit fuels
frs_west_gdf_filtered = frs_west_gdf[~facility_flags].copy()
frs_west_gdf_filtered = frs_west_gdf_filtered.drop(columns=['has_existing_id'])

# Add inWECC column
in_wecc_series = gpd.sjoin(
    frs_west_gdf_filtered, wecc_zones[['geometry']], how='left', predicate='intersects'
)['index_right'].notna()

frs_west_gdf_filtered['inWECC'] = in_wecc_series.values

# Keep relevant columns
frs_west_gdf_filtered['naics_code'] = frs_west_gdf_filtered['naics_code'].fillna('').astype(str)
frs_west_gdf_filtered = frs_west_gdf_filtered[['geometry', 'registry_id', 'primary_name', 'latitude83', 'longitude83', 'naics_code', 'inWECC']]

# Save facilities by sector
extra_frs_facilities_output_path = output_path / 'extra_epa_frs_facilities_west'
extra_frs_facilities_output_path.mkdir(parents=True, exist_ok=True)

for sector, naics_list in sector_to_naics.items():
    extra_facilities_by_sector = pd.DataFrame()
    
    for naics in naics_list:
        mask = frs_west_gdf_filtered['naics_code'].apply(lambda x: str(x).startswith(str(naics)))
        extra_facilities_by_sector = pd.concat([extra_facilities_by_sector, frs_west_gdf_filtered[mask]])
    
    # Save GeoPackage
    extra_facilities_by_sector.to_file(extra_frs_facilities_output_path / f'{sector}_facilities.gpkg', driver='GPKG')

    # Save CSV
    extra_facilities_by_sector.drop(columns=['geometry']).to_csv(
        extra_frs_facilities_output_path / f'{sector}_facilities.csv', index=False
    )